In [ ]:
import sys
import os
# For Jupyter notebooks, use getcwd() instead of __file__
sys.path.append(os.path.join(os.getcwd(), '..'))

from model_trainers.trainer import predict, evaluate
from dataloaders.dataloading import *
from util.search_util import *
from dataloaders.custom_dataloader import *
from models.model_mlp import *
from models.model_small import *
from dataloaders.get_all_samples import *
from dataloaders.imbalanced_dataloader import *
from masks.mask import *

import torch.nn as nn
from torch import optim
import os
import time
import argparse
import wandb
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

/Users/arghasreebanerjee/opt/anaconda3/envs/XAI_week1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset = 'MNIST'
directory = "/Users/arghasreebanerjee/PycharmProjects/BGD"
batch_size = 100
random_seed = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
learning_rate = 0.001
weight_decay = 0.0001
lr_decay = 0.1
no_model = False
model_directory = directory + f'/models/MLP/model_weights_mnist.pth'
dataset_directory = directory + f'/MNIST'
incorrect_loader_path = f'{directory}/dataloaders/MLP/MNIST_incorrect_loader.pth'
avg_mask_dir =f'{directory}/masks/avg_mask_params.pth'

In [6]:
train_loader, valid_loader, test_loader = data_loader_MNIST(directory=directory,
                                                            batch_size=batch_size,
                                                            random_seed=random_seed,
                                                            small=False,
                                                            shuffle=False)

Total batches = 600


In [10]:
incorrect_loader = torch.load(incorrect_loader_path, map_location=torch.device(device))
        
image, _ = next(iter(train_loader))
model = MLP(num_classes=10, input_size=image.size()[1:])
model = model.to(device)
model.load_state_dict(torch.load(model_directory, map_location=torch.device(device))) # map_loaction for running on local machine

mask = Mask(model, random_seed=0, initial_type='uniform')
mask = mask.to(device)
avg_mask_params = torch.load(avg_mask_dir, map_location=torch.device(device))
mask.mask_parameters = nn.ParameterList(nn.Parameter(p, requires_grad=True) for p in avg_mask_params)
        

In [ ]:
def loader_performance(loaders, model, epoch):  # Always assumes you are passing test and incorrect loader in that order
    names = ['train_loader','test_loader', 'incorrect_loader']
    
    for loader, name in zip(loaders, names):
        y_true = []
        y_hat = []
        for img, label in loader:
            img = img.to(device)
            label = label.to(device)

            output = model(img)
            y_pred = torch.argmax(output, dim=1)
            # print(y_pred)
            # return None
             # Convert tensors to lists for comparison
            y_true.extend(label.cpu().numpy().tolist())
            y_hat.extend(y_pred.cpu().numpy().tolist())
        
        # Convert lists to numpy arrays for comparison
        y_true = np.array(y_true)
        y_hat = np.array(y_hat)
        
        # Calculate accuracy
        acc = np.mean(y_true == y_hat)
        if name == 'incorrect_loader':
            c=np.sum(y_true == y_hat)
            print(f'Number of samples correctly predicted is {c}/{len(loader)}')
        print(f"Accuracy of {name} is {acc} at epoch {epoch}")


In [ ]:
loader_performance([train_loader, test_loader, incorrect_loader], mask, 'Final')

Accuracy of train_loader is 0.9285185185185185 at epoch 0
Accuracy of test_loader is 0.9239 at epoch 0
Number of samples correctly predicted is 100/710
Accuracy of incorrect_loader is 0.14084507042253522 at epoch 0
